### Group Members:
### 1) Name: Bhaven Chetan Naik, Student Number: 202004463, STFX Email: x2020ety@stfx.ca
### 2) Name: Dhairya Kishore Bhadra, Student Number: 202005806, STFX Email: x2020flc@stfx.ca

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np

In [2]:
# Loading Dataset
chemicals = pd.read_csv("mapped_features_train.csv") # csv containing mapped training features
test_chemicals = pd.read_csv("mapped_features_test.csv") # csv containing mapped test features

In [3]:
# Visualizing Chemicals Data
chemicals.head()

,Id,Expected,Chemical_Id,Assay_Id,V2,V3,V4,V5,V6,V7,...,V1066,V1067,V1068,V1069,V1070,V1071,V1072,V1073,V1074,V1075
0,2971-36-0;1644,2,2971-36-0,1644,76302,315.982463,4.592,40.46,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2971-36-0;1630,2,2971-36-0,1630,76302,315.982463,4.592,40.46,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2971-36-0;29,2,2971-36-0,29,76302,315.982463,4.592,40.46,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2971-36-0;1618,2,2971-36-0,1618,76302,315.982463,4.592,40.46,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2971-36-0;1638,2,2971-36-0,1638,76302,315.982463,4.592,40.46,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Visualizing testing data of chemicals
test_chemicals.head()

,x,Chemical_Id,Assay_Id,V2,V3,V4,V5,V6,V7,V8,...,V1066,V1067,V1068,V1069,V1070,V1071,V1072,V1073,V1074,V1075
0,88-60-8;1682,88-60-8,1682,6937,164.120115,3.659,20.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,88-60-8;2451,88-60-8,2451,6937,164.120115,3.659,20.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,88-60-8;2442,88-60-8,2442,6937,164.120115,3.659,20.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,88-60-8;32,88-60-8,32,6937,164.120115,3.659,20.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,88-60-8;1382,88-60-8,1382,6937,164.120115,3.659,20.23,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Replacing infinite values of column V15 with their mean values
chemicals['V15'].replace([np.inf, -np.inf], np.nan, inplace=True) # Replacing infinite values with NaN
test_chemicals['V15'].replace([np.inf, -np.inf], np.nan, inplace=True) # Replacing NaN values with mean values
# Repeating the process for test data
chemicals['V15'] = chemicals['V15'].fillna(chemicals['V15'].mean())
test_chemicals['V15'] = test_chemicals['V15'].fillna(test_chemicals['V15'].mean())

In [6]:
# Choosing sample dataset with features used to train the model
X = chemicals.iloc[:, 3:]

y = chemicals['Expected']

In [7]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))

Original dataset shape Counter({2: 66495, 1: 10918})


In [8]:
# Performing Undersampling on the imbalanced data using imbalanced-learn API
from imblearn.under_sampling import TomekLinks
undersample = TomekLinks(sampling_strategy='auto')
X, y  = undersample.fit_resample(X, y)

In [9]:
print('Resampled dataset shape {}'.format(Counter(y)))

Resampled dataset shape Counter({2: 63396, 1: 10918})


In [10]:
# Splitting the sampled data into train and test sets
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Creating a pipeline to perform Feature Selection and Classification using SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = Pipeline([
  ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=0))),
  ('classification',  AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6), n_estimators=600, random_state=0, learning_rate=1.6,
    algorithm="SAMME"))
])

In [12]:
# Training the model
model = clf.fit(X_train, y_train)

In [13]:
# Prediction on X_test
Predicted_Chemicals = model.predict(X_test)

In [14]:
# Accuracy score of the model
model.score(X_test, y_test)

0.9076902375025231

In [15]:
# Internal evaluation F1 score (macro)
from sklearn.metrics import f1_score
f1_score(y_test, Predicted_Chemicals, average='macro')

0.8029974705043526

In [16]:
# Predicting results on Testing data
inp = test_chemicals.iloc[:, 2:]

result = model.predict(inp)

In [17]:
# Generate csv of predicted data
result_csv = {'Id': test_chemicals.x, 'Predicted':result}
pd.DataFrame(result_csv).to_csv('submission_1.csv',index=False)